# CLIMADA

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import box
from shapely.geometry import Point

from climada.hazard import TropCyclone
from climada.entity import Exposures
from climada.entity.exposures import LitPop
from climada.entity.impact_funcs import ImpactFuncSet, ImpfTropCyclone

import adjusted_year_loss_tables as aylt

In [ ]:
## set parameters
# Exposure
my_country = 'AUS'
m, n = 1,1 # based on analysis by Eberenz et al. (2020), (1,1) is the best combination.
my_fin_mode = 'pc'
ref_year = 2023

# Hazard
start_year = 1980
end_year = 2023

my_res_arcsec = 600
starting_phase = 'Nina'

## Exposure

In [ ]:
# Exposure
exp1 = LitPop.from_countries(my_country, my_res_arcsec, (m,n), my_fin_mode, reference_year = ref_year)
exp1.check()

# Remove Lord Howe Island and Macquarie Island
exp = Exposures(exp1.gdf[exp1.geometry.x <= 155])
exp.check()

In [ ]:
exp_with_state = aylt.assign_state_to_exposure(exp.data, "STE_2021_AUST_SHP_GDA2020/STE_2021_AUST_GDA2020.shp").replace("Australian Capital Territory", "New South Wales").replace("New South Wales", "New South Wales and\nAustralian Capital Territory")

In [ ]:
# Move the grid cells from being centroids to actual grid cells

# 2) pull out the raw x/y coordinates of the centroids
xs = np.sort(np.unique(exp_with_state.geometry.x))
ys = np.sort(np.unique(exp_with_state.geometry.y))

# 3) infer grid spacing as the median of the successive differences
dx = np.median(np.diff(xs))
dy = np.median(np.diff(ys))

# 4) build a square polygon around each point
#    using half the resolution in each direction
hx, hy = dx/2, dy/2

def centroid_to_cell(pt):
    x, y = pt.x, pt.y
    return box(x-hx, y-hy, x+hx, y+hy)

exp_with_state["geometry"] = exp_with_state.geometry.apply(centroid_to_cell)


In [ ]:
exp_with_state

In [ ]:
regions = exp_with_state["STE_NAME21"]
regions

In [ ]:
import cartopy.feature as cfeature
states_provinces = cfeature.NaturalEarthFeature(
    category='cultural',
    name='admin_1_states_provinces_lines',
    scale='50m',
    facecolor='none')

In [ ]:
# 1) load your exp_with_state (points) and state polygons
#    (reuse whatever path you used in assign_state_to_exposure)
state_gdf = gpd.read_file("STE_2021_AUST_SHP_GDA2020/STE_2021_AUST_GDA2020.shp").iloc[:-1]  # drop “Other Territories”
state_gdf = state_gdf.to_crs(exp_with_state.crs)

### Plot regions

In [ ]:
import matplotlib.patches as mpatches
import cartopy.crs as ccrs
import matplotlib.colors as mcolors
# import matplotlib.cm as cm

figsize = (6,10)

categories = sorted(regions.unique())
cmap = plt.get_cmap('tab20')
norm = mcolors.Normalize(vmin = 0, vmax = len(categories)-1)
region_color_map = {region:cmap(norm(i)) for i, region in enumerate(categories)}

patches = [mpatches.Patch(color = region_color_map[reg], label = reg) for reg in categories]

fig, axis = plt.subplots(1,1, figsize = figsize, subplot_kw=dict(projection=ccrs.PlateCarree()))
xmin, ymin, xmax, ymax = (
	exp.longitude.min()-1,
	exp.latitude.min()-1,
	exp.longitude.max()+1,
	exp.latitude.max()+1,
)
axis.set_extent((xmin, xmax, ymin, ymax), crs=ccrs.PlateCarree())
axis.add_feature(states_provinces, edgecolor='black');
axis.coastlines()

exp_with_state.plot(column = "STE_NAME21", legend = False, figsize = figsize, cmap = 'tab20', ax=axis)
axis.set_aspect("equal")

fig.legend(
    handles = patches, 
    bbox_to_anchor = (1,0.5), 
    loc="center left",
)

axis.set_xticks([])
axis.set_yticks([])
for spine in axis.spines.values():
    spine.set_visible(False)

fig.tight_layout()
plt.show();

In [ ]:
def format_lon(lon):
    """
    Format longitude for maps that cross the dateline,
    assuming the map is centered at 180° (i.e., central_longitude=180).
    """
    # Convert to [-180, 180] range
    lon = ((lon + 180) % 360) - 180

    if lon == 0:
        return "0°"
    elif lon > 0:
        return f"{abs(lon)}°E"
    else:
        return f"{abs(lon)}°W"

def format_lat(lat):
    if lat == 0: 
        return "0°"
    elif lat > 0: 
        return f"{abs(lat)}°N"
    else:
        return f"{abs(lat)}°S"

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import cartopy.feature as cfeature

projection = ccrs.PlateCarree(central_longitude=180)
fig, axis = plt.subplots(figsize = (13,9),
						 subplot_kw={"projection": projection})
axis.set_extent([33, 233, -50, 0], crs=ccrs.PlateCarree())

# CUSTOM TICKS
xticks = [40,60,80,100,120,140,160,180,-160,-140]
yticks = [-50,-40,-30,-20,-10,0]

# Set ticks with correct transform
axis.set_xticks(xticks, crs=ccrs.PlateCarree())
axis.set_yticks(yticks, crs=ccrs.PlateCarree())

# Set custom tick labels (optional)
axis.set_xticklabels([format_lon(x) for x in xticks], fontsize = 16)
axis.set_yticklabels([format_lat(y) for y in yticks], fontsize = 16)

# Add gridlines (optional)
axis.gridlines(draw_labels=False, linestyle = "--", color='gray', alpha=0.5,
crs = ccrs.PlateCarree(), xlocs = xticks, ylocs = yticks)
axis.gridlines(draw_labels=False, color='red', alpha=0.5,
crs = ccrs.PlateCarree(), xlocs = [75,170], ylocs = [])

axis.add_feature(cfeature.BORDERS)
axis.add_feature(cfeature.COASTLINE);
plt.savefig(f"Figures/map.png", dpi = 300, bbox_inches = "tight")

In [ ]:
import matplotlib.patches as mpatches
import cartopy.crs as ccrs
import matplotlib.colors as mcolors

figsize = (6,10)

fig, axis = plt.subplots(1,1, figsize = figsize)

exp_with_state.plot(column = "STE_NAME21", legend = False, figsize = figsize, cmap = 'tab20', ax=axis)

xmin, ymin, xmax, ymax = (
	exp.longitude.min()-1,
	exp.latitude.min()-1,
	exp.longitude.max()+1,
	exp.latitude.max()+1,
)

state_gdf.plot(figsize=(7,10), edgecolor='black', facecolor="none", ax = axis)
plt.gca().set_xlim(xmin, xmax)
plt.gca().set_ylim(ymin, ymax);
axis.set_aspect("equal")

categories = sorted(regions.unique())
cmap = plt.get_cmap('tab20')
norm = mcolors.Normalize(vmin = 0, vmax = len(categories)-1)
region_color_map = {region:cmap(norm(i)) for i, region in enumerate(categories)}

patches = [mpatches.Patch(color = region_color_map[reg], label = reg) for reg in categories]

fig.legend(
    handles = patches, 
    bbox_to_anchor = (1,0.5), 
    loc="center left",
    fontsize = 14,
)

axis.set_xticks([])
axis.set_yticks([])
for spine in axis.spines.values():
    spine.set_visible(False)

plt.tight_layout()
plt.savefig(f"Figures/aus_regions.png", dpi = 300, bbox_inches = "tight")

In [ ]:
state_exposures = exp_with_state.groupby("STE_NAME21")["value"].sum()
region_thresholds = dict(state_exposures * 1e-6)
region_thresholds

In [ ]:
# check
print(f'Total exposure value to {my_country}: USD {exp.gdf.value.sum():,.2f} / AUD {exp.gdf.value.sum()/0.6630:,.2f} ({ref_year})')

## Hazard

In [ ]:
# Hazard
n_synth_tracks = 1000
haz = TropCyclone.from_hdf5(f'Hazards/haz_aus_{n_synth_tracks}synth.hdf5')
haz.check()
exp.assign_centroids(haz)

In [ ]:
enso_phases = aylt.enso_phases_of_historical_tracks(haz)

In [ ]:
exp.gdf

## Impact

In [ ]:
impf_tc = ImpfTropCyclone.from_emanuel_usa()
impf_set = ImpactFuncSet([impf_tc])
impf_set.check()

In [ ]:
loss_catalogues = aylt.compute_loss_catalogue(haz, exp, impf_set, save_catalogue=False)

In [ ]:
loss_catalogues["No adjustment"].shape

In [ ]:
loss_catalogues["No adjustment"].sum(axis=1)

In [ ]:
exp_with_state

In [ ]:
ax = state_gdf.plot(figsize=(7,10), edgecolor='black', facecolor="none") #, xlim=(minx, maxx)) #, ylim=(miny, maxy))
ax.set_aspect("equal")
plt.gca().set_xlim(xmin, xmax)
plt.gca().set_ylim(ymin, ymax);

In [ ]:
max_loss_each_location = np.array(np.stack([cat.max(axis=0).todense() for cat in loss_catalogues.values()]).max(axis=0)).flatten()
max_loss_each_location.shape

In [ ]:
mean_loss_each_location = np.array(loss_catalogues["No adjustment"].mean(axis=0)).flatten()
mean_loss_each_location.shape

In [ ]:
loss_catalogues["No adjustment"]

In [ ]:
np.mean(max_loss_each_location == 0)

In [ ]:
np.log10(max_loss_each_location)

In [ ]:
exp_with_state["Max Loss"] = np.log10(max_loss_each_location)
exp_with_state["Max Loss"] = exp_with_state["Max Loss"].replace(-np.inf, np.nan)

exp_with_state["Mean Loss"] = mean_loss_each_location

# exp_with_state["Max Loss"] = (max_loss_each_location)
# exp_with_state["Max Loss"] = exp_with_state["Max Loss"].replace(0, np.nan)

exp_with_state["Loss is Possible"] = max_loss_each_location > 0
exp_with_state["Exposure is Positive"] = exp_with_state["value"] > 0

In [ ]:
exp_with_state

In [ ]:
exp_with_state.groupby("STE_NAME21")["Mean Loss"].sum()

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, ax = plt.subplots(1,1, figsize=(7,10))

collection = exp_with_state.plot(
    column="Max Loss", 
    legend=False,
    ax=ax,
    edgecolor="none", 
)

cbar_ax = make_axes_locatable(ax).append_axes("right", size = "6.5%", pad = 0.5)
cbar = fig.colorbar(collection.collections[0], label="value (log10)", cax=cbar_ax)
cbar.ax.tick_params(labelsize = 13)
cbar.set_label("value (log10)", fontsize = 13)

# ax.set_title("Max Observed Loss per Location (log10 USD)", fontsize=16)
ax.set_axis_off()
state_gdf.plot(ax=ax, color="none", edgecolor="black")

ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
ax.set_aspect("equal")

plt.tight_layout()
plt.savefig(f"Figures/max_loss_observed_{n_synth_tracks}synth.png", dpi=300, bbox_inches="tight")

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(7,10))
exp_with_state.plot(
    column="Exposure is Positive", 
    cmap="Pastel1",
    legend=True,
    ax=ax,
    edgecolor="none"
)
ax.set_title("Grid Cells where Exposure is > $0")
ax.set_axis_off()
ax.set_aspect("equal")
state_gdf.plot(ax=ax, color="none", edgecolor="black")

ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
ax.set_aspect("equal")

plt.tight_layout()
plt.savefig(f"Figures/exposure_positive_{n_synth_tracks}.png", dpi=300, bbox_inches="tight")


In [ ]:
# 1) compute mask & coords
mask = (haz.intensity.max(axis=0) > haz.intensity_thres).todense().A1
lons, lats = haz.centroids.coord[:,1], haz.centroids.coord[:,0]

# 2) build a pandas DataFrame
df = pd.DataFrame({
    'lon': lons,
    'lat': lats,
    'above_thr': mask
})

# 3) turn into a GeoDataFrame
gdf_pts = gpd.GeoDataFrame(
    df,
    geometry=[Point(x,y) for x,y in zip(df['lon'], df['lat'])],
    crs="EPSG:4326"
)

# 4) plot states + points
fig, ax = plt.subplots(figsize=(7,10))


# point‐choropleth keyed on 'above_thr'
gdf_pts.plot(
    ax=ax,
    column='above_thr',
    categorical=True,
    cmap='Pastel1',
    markersize=5,
    legend=True,
    legend_kwds={'loc':'upper right', 'title':'> threshold'}
)

# state boundaries
state_gdf.plot(ax=ax, facecolor="none", edgecolor="black", linewidth=1)

ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
ax.set_aspect("equal")
ax.set_axis_off()
ax.set_title("Grid cells where max intensity > threshold (17.5 m/s)")
plt.tight_layout()
plt.savefig(f"Figures/hazard_above_threshold_{n_synth_tracks}.png", dpi=300, bbox_inches="tight")


In [ ]:
haz.centroids.coord.shape

In [ ]:
# 1) compute mask & coords
lons, lats = haz.centroids.coord[:,1], haz.centroids.coord[:,0]

# 2) build a pandas DataFrame
df = pd.DataFrame({
    'lon': lons,
    'lat': lats,
    'max_intensity': haz.intensity.max(axis=0).todense().A1,
})

# 3) turn into a GeoDataFrame
gdf_pts = gpd.GeoDataFrame(
    df,
    geometry=[Point(x,y) for x,y in zip(df['lon'], df['lat'])],
    crs="EPSG:4326"
)

# 4) plot states + points
fig, ax = plt.subplots(figsize=(7,10))


# point‐choropleth keyed on 'above_thr'
collection = gdf_pts.plot(
    ax=ax,
    column='max_intensity',
    # categorical=True,
    # cmap='Pastel1',
    markersize=5,
	vmin = 0,
	vmax = 75,
)
# Add a colorbar
sm = gdf_pts.plot(column='max_intensity', ax=ax, legend=False, markersize=5)
# sm.set_clim(0, 30)
# sm.set_cmap('viridis')

cbar_ax = make_axes_locatable(ax).append_axes("right", size = "6.5%", pad = 0.1)
cbar = fig.colorbar(collection.collections[0], label="Intensity (m/s)", cax=cbar_ax)
cbar.ax.tick_params(labelsize = 13)
cbar.set_label("Intensity (m/s)", fontsize = 13)

# state boundaries
state_gdf.plot(ax=ax, facecolor="none", edgecolor="black", linewidth=1)

ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
ax.set_aspect("equal")
ax.set_axis_off()
# ax.set_title("Grid cells max intensity")
plt.tight_layout()
plt.savefig(f"Figures/hazard_max_intensity_{n_synth_tracks}.png", dpi=300, bbox_inches="tight")


In [ ]:
fig, ax = plt.subplots(1,1, figsize=(7,10))
exp_with_state.plot(
    column="Loss is Possible", 
    cmap="Pastel1",
    legend=True,
    ax=ax,
    edgecolor="none"
)

state_gdf.plot(ax=ax, color="none", edgecolor="black")

ax.set_title("Grid cells where losses are ever observed")
ax.set_axis_off()

ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
ax.set_aspect("equal")

plt.tight_layout()
plt.savefig(f"Figures/loss_observed_{n_synth_tracks}synth.png", dpi=300, bbox_inches="tight")

In [ ]:
# ENSO phases for each track
enso_phases = aylt.enso_phases_of_historical_tracks(haz)
phases = sorted(set(enso_phases))

# Get coordinates
lons, lats = haz.centroids.coord[:, 1], haz.centroids.coord[:, 0]

# Step 1: Compute global vmin and vmax across all phases
all_max_vals = []
for phase in phases:
    idx = [i for i, p in enumerate(enso_phases) if p == phase]
    if not idx:
        continue
    max_vals = haz.intensity[idx].max(axis=0).todense().A1
    all_max_vals.append(max_vals)

# Stack all phase max_vals and compute overall min/max
all_max_vals_stacked = np.vstack(all_max_vals)
global_vmin = all_max_vals_stacked.min()
global_vmax = all_max_vals_stacked.max()

print(f"Global vmin: {global_vmin:.2f}, vmax: {global_vmax:.2f}")

# Step 2: Plot with consistent vmin/vmax
for phase in phases:
    idx = [i for i, p in enumerate(enso_phases) if p == phase]
    if not idx:
        print(f"No tracks for {phase}, skipping...")
        continue

    max_vals = haz.intensity[idx].max(axis=0).todense().A1

    df = pd.DataFrame({
        'lon': lons,
        'lat': lats,
        'max_intensity': max_vals,
    })
    gdf_pts = gpd.GeoDataFrame(
        df,
        geometry=[Point(x, y) for x, y in zip(df['lon'], df['lat'])],
        crs="EPSG:4326"
    )

    fig, ax = plt.subplots(figsize=(5, 4))
    gdf_pts.plot(
        ax=ax,
        column='max_intensity',
        cmap='viridis',
        markersize=5,
        legend=False,
        vmin=global_vmin,
        vmax=global_vmax,
        legend_kwds={"label": "Intensity (m/s)"},
    )

    cbar_ax = make_axes_locatable(ax).append_axes("right", size = "6.5%", pad = 0.1)
    cbar = fig.colorbar(collection.collections[0], label="Intensity (m/s)", cax=cbar_ax)
    cbar.ax.tick_params(labelsize = 10)
    cbar.set_label("Intensity (m/s)", fontsize = 10)

    xmin, ymin, xmax, ymax = gdf_pts.total_bounds
    bbox_polygon = box(xmin, ymin, xmax, ymax)
    state_gdf_clipped = gpd.clip(state_gdf, bbox_polygon)
    state_gdf_clipped.plot(ax=ax, facecolor="none", edgecolor="black", linewidth=1)

    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)
    ax.set_aspect("equal")
    ax.set_axis_off()

    # ax.set_title(f"Max intensity by location\nENSO phase: {phase}")
    plt.tight_layout()
    plt.savefig(f"Figures/hazard_max_intensity_{n_synth_tracks}_{phase}.png", dpi=300, bbox_inches="tight")
    # plt.close()


In [ ]:
haz_enso = aylt.enso_phases_of_historical_tracks(haz)
v_min = haz.intensity.min()
v_max = haz.intensity.max()

event_ids_Nina = haz.event_id[haz_enso == "Nina"]
event_ids_Neutral = haz.event_id[haz_enso == "Neutral"]
event_ids_Nino = haz.event_id[haz_enso == "Nino"]

haz_Nina = haz.select(event_id = event_ids_Nina)
haz_Neutral = haz.select(event_id = event_ids_Neutral)
haz_Nino = haz.select(event_id = event_ids_Nino)

ax = haz.plot_intensity(0, adapt_fontsize = True, figsize = (7,10), shapes = False, vmin = v_min, vmax = v_max)
ax.set_title("")
ax.add_feature(states_provinces, edgecolor='black')
ax.coastlines();
for spine in ax.spines.values():
	spine.set_visible(False)

# In hazard.util.plot make_map(), set draw_labels = False, alpha = 0

In [ ]:

# 2) aggregate the point‐level flag up to each state
#    this gives True if *any* location in that state has a possible loss
state_flag = (
    exp_with_state
    .groupby("STE_NAME21")["Loss is Possible"]
    .max()                         # bool max = any(True)
    .rename("Loss is Possible")
)

# 3) join it onto the state polygons
state_gdf = state_gdf.join(state_flag, on="STE_NAME21")
state_gdf["Loss is Possible"] = state_gdf["Loss is Possible"].fillna(False)

# 4) plot only the polygons, coloured by the flag
ax = state_gdf.plot(
    column="Loss is Possible",
    categorical=True,            # ensures two discrete colours
    cmap="Pastel1",
    legend=True,
    edgecolor="black",
    linewidth=0.5,
    figsize=(7,10),
)

ax.set_title("States where loss is observed")
ax.axis("off")
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
ax.set_aspect("equal")
plt.tight_layout()
plt.savefig(f"Figures/loss_observed_states_{n_synth_tracks}.png", dpi=300, bbox_inches="tight")
# plt.show()


In [ ]:
import rasterio
from mpl_toolkits.axes_grid1 import make_axes_locatable
import climada.util.plot as u_plot
import numpy as np
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import geopandas as gpd
from climada.util.coordinates import pts_to_raster_meta, latlon_bounds, get_resolution
from climada.util.plot import make_map

axis = None
figsize = (5,4)
fill = True
raster_f = lambda x: np.log10((np.fmax(x + 1, 1)))
points_df = exp.data
res = None
raster_res = None
DEF_CRS = exp.crs
proj_data = ccrs.PlateCarree()
adapt_fontsize = False

val_names = ["value"]

if "geometry" in points_df:
	latval = points_df.geometry.y
	lonval = points_df.geometry.x
else:
	latval = points_df["latitude"].values
	lonval = points_df["longitude"].values

if not res:
	res = np.abs(get_resolution(latval, lonval)).min()
if not raster_res:
	raster_res = res

fun = lambda r: r.geometry.buffer(res/2).envelope

def apply_box(df_exp):
	return df_exp.apply(fun, axis=1)

df_poly = gpd.GeoDataFrame(points_df[val_names])
df_poly["_-geometry-prov"] = apply_box(points_df)

df_poly.set_geometry(
	"_-geometry-prov",
	crs=exp.crs,
	inplace=True,
	drop=True,
)

# renormalize longitude if necessary
xmin, ymin, xmax, ymax = latlon_bounds(latval, lonval)
x_mid = 0.5 * (xmin + xmax)
# we don't really change the CRS when rewrapping, so we reset the CRS attribute afterwards
df_poly = df_poly.to_crs({"proj": "longlat", "lon_wrap": x_mid}).set_crs(
    DEF_CRS, allow_override=True
)

# construct raster
rows, cols, ras_trans = pts_to_raster_meta(
    (xmin-1, ymin-1, xmax+1, ymax+1), (raster_res, -raster_res)
)
raster_out = np.zeros((len(val_names), rows, cols))

# TODO: parallel rasterize
for i_val, val_name in enumerate(val_names):
    raster_out[i_val, :, :] = rasterio.features.rasterize(
        list(zip(df_poly.geometry, df_poly[val_name])),
        out_shape=(rows, cols),
        transform=ras_trans,
        fill=0,
        all_touched=True,
        dtype=rasterio.float32,
    )

meta = {
    "crs": df_poly.crs,
    "height": rows,
    "width": cols,
    "transform": ras_trans,
}

raster, meta = raster_out, meta

raster = raster.reshape((meta["height"], meta["width"]))

xmin, ymin, xmax, ymax = (
	exp.longitude.min()-1,
	exp.latitude.min()-1,
	exp.longitude.max()+1,
	exp.latitude.max()+1,
)
# Plot #########################################################################
proj_plot = ccrs.PlateCarree()

_, axis, fontsize = make_map(
	figsize = figsize,
	proj = proj_data,
	adapt_fontsize = adapt_fontsize)

cbar_ax = make_axes_locatable(axis).append_axes(
	"right", size="6.5%", pad=0.1, axes_class=plt.Axes
)
axis.set_extent((xmin, xmax, ymin, ymax), crs=proj_plot)
axis.set_aspect("equal")

u_plot.add_shapes(axis)

if not fill:
	raster = np.where(raster == 0, np.nan, raster)
	raster_f = lambda x: np.log10((np.maximum(x + 1, 1)))

imag = axis.imshow(
	raster_f(raster),
	origin="upper",
	extent=(xmin, xmax, ymin, ymax),
	transform=proj_data,
	cmap = 'magma',
)
cbar = plt.colorbar(imag, label="value (log10)", cax=cbar_ax)
plt.tight_layout()
plt.draw()
if fontsize:
	cbar.ax.tick_params(labelsize=fontsize)
	cbar.ax.yaxis.get_offset_text().set_fontsize(fontsize)
	for item in [axis.title, cbar.ax.xaxis.label, cbar.ax.yaxis.label]:
		item.set_fontsize(fontsize)

axis.add_feature(states_provinces, edgecolor='dimgray');
plt.savefig(f"Figures/exposure.png", dpi = 300, bbox_inches = "tight")

In [ ]:
xmin, xmax, ymin, ymax

In [ ]:
import rasterio
from mpl_toolkits.axes_grid1 import make_axes_locatable
import climada.util.plot as u_plot
import numpy as np
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import geopandas as gpd
from climada.util.coordinates import pts_to_raster_meta, latlon_bounds, get_resolution
from climada.util.plot import make_map

axis = None
figsize = (5,4)
fill = True
raster_f = lambda x: np.log10((np.fmax(x + 1, 1)))
points_df = exp.data
res = None
raster_res = None
DEF_CRS = exp.crs
proj_data = ccrs.PlateCarree()
adapt_fontsize = False

val_names = ["value"]

latval = points_df.geometry.y
lonval = points_df.geometry.x

res = np.abs(get_resolution(latval, lonval)).min()
raster_res = res

fun = lambda r: r.geometry.buffer(res/2).envelope

def apply_box(df_exp):
	return df_exp.apply(fun, axis=1)

df_poly = gpd.GeoDataFrame(points_df[val_names])
df_poly["_-geometry-prov"] = apply_box(points_df)

df_poly.set_geometry(
	"_-geometry-prov",
	crs=exp.crs,
	inplace=True,
	drop=True,
)

# renormalize longitude if necessary
xmin, ymin, xmax, ymax = latlon_bounds(latval, lonval)
x_mid = 0.5 * (xmin + xmax)
# we don't really change the CRS when rewrapping, so we reset the CRS attribute afterwards
df_poly = df_poly.to_crs({"proj": "longlat", "lon_wrap": x_mid}).set_crs(
    DEF_CRS, allow_override=True
)

# construct raster
rows, cols, ras_trans = pts_to_raster_meta(
    (xmin-1, ymin-1, xmax+1, ymax+1), (raster_res, -raster_res)
)
raster_out = np.zeros((len(val_names), rows, cols))

# TODO: parallel rasterize
for i_val, val_name in enumerate(val_names):
    raster_out[i_val, :, :] = rasterio.features.rasterize(
        list(zip(df_poly.geometry, df_poly[val_name])),
        out_shape=(rows, cols),
        transform=ras_trans,
        fill=0,
        all_touched=True,
        dtype=rasterio.float32,
    )

meta = {
    "crs": df_poly.crs,
    "height": rows,
    "width": cols,
    "transform": ras_trans,
}

raster, meta = raster_out, meta

raster = raster.reshape((meta["height"], meta["width"]))

xmin, ymin, xmax, ymax = (
	exp.longitude.min()-1,
	exp.latitude.min()-1,
	exp.longitude.max()+1,
	exp.latitude.max()+1,
)

exp_geom_qld = exp.gdf[regions== "Queensland"].geometry
xmin_qld, ymin_qld, xmax_qld, ymax_qld = (
	exp_geom_qld.x.min()-0.5,
	exp_geom_qld.y.min()-0.5,
	exp_geom_qld.x.max()+0.5,
	exp_geom_qld.y.max()+0.5
)
# Plot #########################################################################
proj_plot = ccrs.PlateCarree()

_, axis, fontsize = make_map(
	figsize = figsize,
	proj = proj_data,
	adapt_fontsize = adapt_fontsize)

cbar_ax = make_axes_locatable(axis).append_axes(
	"right", size="6.5%", pad=0.1, axes_class=plt.Axes
)
axis.set_extent((xmin_qld, xmax_qld, ymin_qld, ymax_qld), crs=proj_plot)
axis.set_aspect("equal")
u_plot.add_shapes(axis)

if not fill:
	raster = np.where(raster == 0, np.nan, raster)
	raster_f = lambda x: np.log10((np.maximum(x + 1, 1)))

imag = axis.imshow(
	raster_f(raster),
	origin="upper",
	extent=(xmin, xmax, ymin, ymax),
	transform=proj_data,
	cmap = 'magma',
)
cbar = plt.colorbar(imag, label="value (log10)", cax=cbar_ax)
plt.tight_layout()
plt.draw()
if fontsize:
	cbar.ax.tick_params(labelsize=fontsize)
	cbar.ax.yaxis.get_offset_text().set_fontsize(fontsize)
	for item in [axis.title, cbar.ax.xaxis.label, cbar.ax.yaxis.label]:
		item.set_fontsize(fontsize)

axis.add_feature(states_provinces, edgecolor='dimgray');
plt.savefig(f"Figures/exposure_qld.png", dpi = 300, bbox_inches = "tight")

In [ ]:
# ENSO phases for each track
enso_phases = aylt.enso_phases_of_historical_tracks(haz)
phases = sorted(set(enso_phases))

# Get coordinates
lons, lats = haz.centroids.coord[:, 1], haz.centroids.coord[:, 0]

# Step 1: Compute global vmin and vmax across all phases
all_max_vals = []
for phase in phases:
    idx = [i for i, p in enumerate(enso_phases) if p == phase]
    if not idx:
        continue
    max_vals = haz.intensity[idx].max(axis=0).todense().A1
    all_max_vals.append(max_vals)

# Stack all phase max_vals and compute overall min/max
all_max_vals_stacked = np.vstack(all_max_vals)
global_vmin = all_max_vals_stacked.min()
global_vmax = all_max_vals_stacked.max()

print(f"Global vmin: {global_vmin:.2f}, vmax: {global_vmax:.2f}")

exp_geom_qld = exp.gdf[regions== "Queensland"].geometry
xmin_qld, ymin_qld, xmax_qld, ymax_qld = (
	exp_geom_qld.x.min()-0.5,
	exp_geom_qld.y.min()-0.5,
	exp_geom_qld.x.max()+0.5,
	exp_geom_qld.y.max()+0.5
)

# Step 2: Plot with consistent vmin/vmax
for phase in phases:
    idx = [i for i, p in enumerate(enso_phases) if p == phase]
    if not idx:
        print(f"No tracks for {phase}, skipping...")
        continue

    max_vals = haz.intensity[idx].max(axis=0).todense().A1

    df = pd.DataFrame({
        'lon': lons,
        'lat': lats,
        'max_intensity': max_vals,
    })
    gdf_pts = gpd.GeoDataFrame(
        df,
        geometry=[Point(x, y) for x, y in zip(df['lon'], df['lat'])],
        crs="EPSG:4326"
    )

    fig, ax = plt.subplots(figsize=(5, 4))
    collection = gdf_pts.plot(
        ax=ax,
        column='max_intensity',
        cmap='viridis',
        markersize=5,
        legend=False,
        vmin=global_vmin,
        vmax=global_vmax,
        legend_kwds={"label": "Intensity (m/s)"},
    )

    cbar_ax = make_axes_locatable(ax).append_axes("right", size = "6.5%", pad = 0.1)
    cbar = fig.colorbar(collection.collections[0], label="Intensity (m/s)", cax=cbar_ax)
    cbar.ax.tick_params(labelsize = 10)
    cbar.set_label("Intensity (m/s)", fontsize = 10)

    xmin, ymin, xmax, ymax = gdf_pts.total_bounds
    bbox_polygon = box(xmin, ymin, xmax, ymax)
    state_gdf_clipped = gpd.clip(state_gdf, bbox_polygon)
    state_gdf_clipped.plot(ax=ax, facecolor="none", edgecolor="black", linewidth=1)

    ax.set_xlim(xmin_qld, xmax_qld)
    ax.set_ylim(ymin_qld, ymax_qld)
    ax.set_aspect("equal")
    ax.set_axis_off()

    # ax.set_title(f"Max intensity by location\nENSO phase: {phase}")
    plt.tight_layout()
    plt.savefig(f"Figures/hazard_max_intensity_{n_synth_tracks}_{phase}_QLD.png", dpi=300, bbox_inches="tight")
    # plt.close()
